In [ ]:
# !pip install selenium

In [21]:
pip install pipreqs

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7389aef30bfc01fc52e3c63fad5f49830dc13004164541d28bdd296750d5a476
  Stored in directory: c:\users\balde\appdata\local\pip\cache\wheels\fc\ab\d4\5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install pipreqs

Note: you may need to restart the kernel to use updated packages.


# Scraping Job info from LinkedIn

<a href="https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba">Tutorial here.</a>


In [ ]:
import os
import time
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from dotenv import load_dotenv


driver_path = "geckodriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

C:\Users\balde\AppData\Local\Temp\ipykernel_23656\3879882641.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


## Authentication

In [ ]:
# Set up environment variables in Powershell:

# $Env:Email = "email"
# echo $Env:Email

In [ ]:
delay = 5

# load the credentials
load_dotenv('credentials.env')

# set credentials:
email = os.getenv('Email')
password = os.getenv('Password')
print(email)

baldebecaye10@gmail.com


In [ ]:
# Go to linkedin and login
driver.get('https://www.linkedin.com/login')

# TODO: Use event listeners
# wait for the browser to load elements
time.sleep(delay)

driver.find_element(By.ID, "username").send_keys(email)
driver.find_element(By.ID, "password").send_keys(password, Keys.ENTER)

## Search jobs by title and location

In [ ]:
job_title = "data science"
job_location = "canada"

In [ ]:
def wait_till_element_ready(by, selector):
    """
    Wait for a given element to be ready

    :param by: the selector strategy. E.g.: By.ID, By.CLASS_NAME, etc.
    :param selector: the CSS selector according to the "By" strategy. E.g.: 'class name', 'id', etc.
    :return:
    """

    colors = {'WARNING': '\033[93m', 'ERROR': '\033[91m'}
    try:
        WebDriverWait(driver, 8).until(EC.presence_of_element_located((by, selector)))
        time.sleep(delay)
    except TimeoutException:
        print(colors['WARNING'] + "Warning: Element has not been loaded!")
        pass

In [ ]:
driver.get("https://www.linkedin.com/jobs/")
# time.sleep(delay)

# find the two search boxes: title and location
wait_till_element_ready(By.CLASS_NAME, "jobs-search-box__text-input")
search_bars = driver.find_elements(By.CLASS_NAME, "jobs-search-box__text-input")
print('Found', len(search_bars), 'search_bars')

# enter job title in the search  by keyword bar
search_by_keyword_bar = search_bars[0]
search_by_keyword_bar.send_keys(job_title)

time.sleep(3)
# enter job location in the search by location bar
search_by_location_bar = search_bars[3]
search_by_location_bar.send_keys(job_location, Keys.RETURN)

Found 5 search_bars


## Get other Pages

In [ ]:
pagination = driver.find_element(By.CLASS_NAME, "artdeco-pagination__pages")
pagination_buttons = pagination.find_elements(By.XPATH, './/button')
# pagination_buttons[1].click()

In [ ]:
pagination_buttons

[<selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="bf439995-b871-4890-80a7-7cafa4d32420")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="fae64dbc-c225-4504-ac6d-ce3fdac367e4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="1655a866-2cdf-4024-b4ad-c910375f3156")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="d65fdff6-7d3a-4ef4-87f7-262d1caf072d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="8cf5b9ec-2a15-4ee2-823d-dcdaccdf47c8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="a91df700-363a-4822-a90f-1e1776075c86")>,
 <selenium.webdriver.remote.webelement.WebElement (session="701205e59b16a69e9fae2e86ae9cf297", element="cb856768-8cce-48ca-88b9-7c

## To Navigate through the pages

## Get the Jobs Info

In [ ]:
def get_job_skills():

    # click display skills popup
    wait_till_element_ready(By.CLASS_NAME, 'jobs-unified-top-card__job-insight-text-button')
    # time.sleep(delay)
    view_skills_button = driver.find_element(By.CLASS_NAME, 'jobs-unified-top-card__job-insight-text-button')
    view_skills_button.click()

    # get skills list
    wait_till_element_ready(By.CLASS_NAME, "job-details-skill-match-status-list")
    # time.sleep(delay)
    skills_list = driver.find_element(By.CLASS_NAME, 'job-details-skill-match-status-list')

    # get clean skills list
    skills_list_clean = skills_list.text.split('\n')
    skills_list_clean = list(filter(lambda skill: skill != 'Add', skills_list_clean))

    # close the skills list popup
    wait_till_element_ready(By.CLASS_NAME, 'artdeco-modal__dismiss')
    # time.sleep(delay)
    close_popup_button = driver.find_element(By.CLASS_NAME, 'artdeco-modal__dismiss')
    close_popup_button.click()

    return skills_list_clean


In [ ]:
# Get the results of the first page
wait_till_element_ready(By.CLASS_NAME, "jobs-search-results__list-item")
# time.sleep(delay)
jobs_list = driver.find_elements(By.CLASS_NAME, "jobs-search-results__list-item")

jobs = []
for job in jobs_list:

    time.sleep(delay)
    # scroll the job div into view
    driver.execute_script("arguments[0].scrollIntoView();", job)
    job.click()
    # time.sleep(delay)

    # get the position company, location, and remote/on-site
    try:
        [position, company, location, work_mode] = job.text.split('\n')[:4]
        break
    except ValueError:
        position = job.text.split('\n')
        [company, location, work_mode] = ''
        print('job.text.split Error', job.text.split('\n'))

    # get skills list for the job
    skills = get_job_skills()

    # description
    description = driver.find_elements(By.CLASS_NAME, "jobs-description-content__text")

    # all the details
    # details = driver.find_element(By.ID, "job-details").text

    # do what you want with that info...
    jobs.append([position, company, location, work_mode, skills, description])

Profile - Profil recherché - About You
Skills - Requirements - Position Requirements - Qualifications - Basic Qualifications - Expérience - Éxigences - Compétences - QUALITÉS
Atouts - Preferred Qualifications - Autres exigences

In [ ]:
jobs[0][4]

'SQL\nCommunication\nAdd\nDrones\nAdd\nFull-Stack Development\nAdd\nHTML\nAdd\nInformation Technology\nAdd\nJavaScript\nAdd\nNetworking\nAdd\nProcedural Programming\nAdd\nTechnical Support\nAdd'

# Connect to SQL Server

In [ ]:
# !pip install pypyodbc

In [ ]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL Server'
SERVER_NAME = '.'
DATABASE_NAME = 'LinkedIn'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={{{SERVER_NAME}}};
    DATABASE={{{DATABASE_NAME}}};
    Trust_Connection=yes;
"""

conn = odbc.connect(connectString=connection_string)
print(conn)

### Fetch jobs

In [ ]:
# # Execute a sample query
# cursor.execute('SELECT * FROM your_table_name')
# # Fetch the results of the query
# rows = cursor.fetchall()
# # Print the results
# for row in rows:
#     print(row)

### Insert jobs into the database

In [ ]:
def save_jobs(jobs):
    """
    Inserts the jobs into the databse

    :param jobs: jobs to be inserted
    :return:
    """
    insert_query = "INSERT INTO Jobs (position, company, location, work_mode, skills, details) VALUES (?, ?, ?, ?, ?, ?)"

    for job in jobs:
        position = job[0]
        company = job[1]
        location = job[2]
        work_mode = job[3]
        skills = job[4]
        details = job[5]

        # transofrm skills array to string
        skills_str = ','.join(skills)

        # Define values to be inserted
        values = (position, company, location, work_mode, skills_str, details)

        # Execute insert statement
        cursor.execute(insert_query, values)

        # Commit changes to the database
        conn.commit()

In [ ]:
# Open Connexion
cursor = conn.cursor()

# save jobs
save_jobs(jobs)

### Close Connection

In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()